In [1]:
from attorch.dataset import ListDataset
from blinkende_lichter.datatools import RandomFlip, TypeConversion 
from torchvision import transforms 
from torch.utils.data import DataLoader 
import numpy as np 
import numpy as np
import torch

from PIL import Image
from argparse import ArgumentParser

from torch.optim import SGD, Adam
from torch.autograd import Variable

from IPython.core.debugger import Tracer


from blinkende_lichter.unet.network import UNet
from blinkende_lichter.unet.criterion import CrossEntropyLoss2d

from random import randint
import matplotlib.pyplot as plt
import random

In [2]:

transform = transforms.Compose([    
    RandomFlip(),    
    TypeConversion(np.float32, np.long) ]) 
dat = np.load('../datasets/data_interpolated.npz') 
train = ListDataset(*dat['train'], transform=transform) 
val = ListDataset(*dat['validation']) 
trainloader = DataLoader( train, shuffle=True, batch_size=1) 
valloader = DataLoader(val, shuffle=True) 
for x, y in trainloader:
    print(x.size(), y.size())
images, labels = next(iter(trainloader))

torch.Size([1, 2, 512, 512]) torch.Size([1, 512, 512])
torch.Size([1, 2, 512, 512]) torch.Size([1, 512, 512])


In [ ]:
NUM_CHANNELS = 2
NUM_CLASSES = 2

Net = UNet
model = Net(NUM_CLASSES)
model = model.cuda()


In [ ]:
model.train()
weight = torch.ones(NUM_CLASSES)
weight[0] = 0      
criterion = CrossEntropyLoss2d()
optimizer = Adam(model.parameters())
epoch_loss = []
for epoch in range(1, 101):
    

    for step, (images, labels) in enumerate(trainloader):
        Tracer()()
        images = images.cuda()
        labels = labels.cuda()

        inputs = Variable(images)
        targets = Variable(labels)
        optimizer.zero_grad()
        outputs = model(inputs)

        loss = criterion(outputs, targets)

        loss.backward()
        optimizer.step()
        epoch_loss.append((loss.data[0]))
torch.save(epoch_loss, "loss")
print("Training complete.")

In [ ]:
labels

In [ ]:
labels.max()

In [ ]:
labels.min()

In [ ]:
labels.type

In [ ]:
labels[0].max()

In [ ]:
labels[0].min()

In [ ]:
len(labels[0])

In [ ]:
labels[0].size()

In [ ]:
len(labels[0][labels[0]==0])

In [ ]:
len(labels[0][labels[0]==255])

In [ ]:
len(labels[0][labels[0]==0]) + len(labels[0][labels[0]==255])

In [ ]:
labels[0][labels[0]!=128]

In [ ]:
test = labels.numpy()

In [ ]:
test

In [ ]:
test[0].shape

In [ ]:
plt.close()

In [ ]:
%matplotlib inline

In [ ]:
plt.imshow(test[0])

In [6]:
arr = train.data[1]

In [14]:
arr[0][arr[0]==255]

array([255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
       255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
       255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
       255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
       255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
       255, 255, 255, 255, 255, 255, 255, 255, 255])